# Mechanizmy pozornosti a transformery

Jednou z hlavných nevýhod rekurentných sietí je, že všetky slová v sekvencii majú rovnaký vplyv na výsledok. To spôsobuje suboptimálny výkon pri štandardných modeloch LSTM encoder-decoder pre úlohy sekvencie na sekvenciu, ako je rozpoznávanie pomenovaných entít alebo strojový preklad. V skutočnosti majú konkrétne slová v vstupnej sekvencii často väčší vplyv na výstupy ako ostatné.

Zvážte model sekvencie na sekvenciu, ako je strojový preklad. Ten je implementovaný pomocou dvoch rekurentných sietí, kde jedna sieť (**encoder**) zhrnie vstupnú sekvenciu do skrytého stavu a druhá (**decoder**) rozvinie tento skrytý stav do preloženého výsledku. Problém s týmto prístupom je, že konečný stav siete má problém zapamätať si začiatok vety, čo vedie k nízkej kvalite modelu pri dlhých vetách.

**Mechanizmy pozornosti** poskytujú spôsob, ako vážiť kontextuálny vplyv každého vstupného vektora na každú výstupnú predikciu RNN. Implementuje sa to vytvorením skratiek medzi medzistavmi vstupnej RNN a výstupnej RNN. Týmto spôsobom, pri generovaní výstupného symbolu $y_t$, zohľadníme všetky skryté stavy vstupu $h_i$, s rôznymi váhovými koeficientmi $\alpha_{t,i}$.

![Obrázok zobrazujúci model encoder/decoder s vrstvou aditívnej pozornosti](../../../../../translated_images/sk/encoder-decoder-attention.7a726296894fb567.webp)
*Model encoder-decoder s mechanizmom aditívnej pozornosti podľa [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citované z [tohto blogového príspevku](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Maticu pozornosti $\{\alpha_{i,j}\}$ môžeme interpretovať ako mieru, do akej konkrétne vstupné slová ovplyvňujú generovanie daného slova vo výstupnej sekvencii. Nižšie je príklad takejto matice:

![Obrázok zobrazujúci vzorové zarovnanie nájdené RNNsearch-50, prevzaté z Bahdanau - arviz.org](../../../../../translated_images/sk/bahdanau-fig3.09ba2d37f202a6af.webp)

*Obrázok prevzatý z [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Obr.3)*

Mechanizmy pozornosti sú zodpovedné za veľkú časť súčasného alebo takmer súčasného stavu techniky v spracovaní prirodzeného jazyka. Pridanie pozornosti však výrazne zvyšuje počet parametrov modelu, čo viedlo k problémom so škálovaním RNN. Kľúčovým obmedzením škálovania RNN je, že rekurentná povaha modelov sťažuje dávkovanie a paralelizáciu tréningu. V RNN musí byť každý prvok sekvencie spracovaný v sekvenčnom poradí, čo znamená, že ho nemožno ľahko paralelizovať.

Prijatie mechanizmov pozornosti v kombinácii s týmto obmedzením viedlo k vzniku dnes už štandardných modelov Transformer, ktoré poznáme a používame, od BERT po OpenGPT3.

## Modely Transformer

Namiesto prenášania kontextu každej predchádzajúcej predikcie do ďalšieho kroku hodnotenia používajú **modely Transformer** **pozíciové kódovania** a pozornosť na zachytenie kontextu daného vstupu v rámci poskytnutého okna textu. Obrázok nižšie ukazuje, ako pozíciové kódovania s pozornosťou dokážu zachytiť kontext v rámci daného okna.

![Animovaný GIF zobrazujúci, ako sa hodnotenia vykonávajú v modeloch Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Keďže každá vstupná pozícia je mapovaná nezávisle na každú výstupnú pozíciu, transformery dokážu lepšie paralelizovať ako RNN, čo umožňuje oveľa väčšie a expresívnejšie jazykové modely. Každá hlava pozornosti môže byť použitá na učenie rôznych vzťahov medzi slovami, čo zlepšuje následné úlohy spracovania prirodzeného jazyka.

**BERT** (Bidirectional Encoder Representations from Transformers) je veľmi veľká viacvrstvová sieť Transformer s 12 vrstvami pre *BERT-base* a 24 pre *BERT-large*. Model je najprv predtrénovaný na veľkom korpuse textových dát (Wikipedia + knihy) pomocou nesupervidovaného tréningu (predikcia maskovaných slov vo vete). Počas predtrénovania model absorbuje významnú úroveň porozumenia jazyka, ktorú je možné následne využiť s inými dátovými súbormi pomocou jemného doladenia. Tento proces sa nazýva **transfer learning**.

![Obrázok z http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sk/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Existuje mnoho variácií architektúr Transformer, vrátane BERT, DistilBERT, BigBird, OpenGPT3 a ďalších, ktoré je možné jemne doladiť. Balík [HuggingFace](https://github.com/huggingface/) poskytuje úložisko na tréning mnohých z týchto architektúr pomocou PyTorch.

## Použitie BERT na klasifikáciu textu

Pozrime sa, ako môžeme použiť predtrénovaný model BERT na riešenie našej tradičnej úlohy: klasifikácie sekvencií. Budeme klasifikovať náš pôvodný dataset AG News.

Najprv načítajme knižnicu HuggingFace a náš dataset:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Keďže budeme používať predtrénovaný model BERT, budeme potrebovať špecifický tokenizer. Najskôr načítame tokenizer spojený s predtrénovaným modelom BERT.

Knižnica HuggingFace obsahuje úložisko predtrénovaných modelov, ktoré môžete použiť jednoducho tým, že ich názvy zadáte ako argumenty funkcií `from_pretrained`. Všetky potrebné binárne súbory pre model sa automaticky stiahnu.

Avšak, v určitých prípadoch budete potrebovať načítať vlastné modely. V takom prípade môžete špecifikovať adresár, ktorý obsahuje všetky relevantné súbory, vrátane parametrov pre tokenizer, súboru `config.json` s parametrami modelu, binárnych váh, atď.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objekt `tokenizer` obsahuje funkciu `encode`, ktorú je možné priamo použiť na kódovanie textu:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Potom vytvorme iterátory, ktoré budeme používať počas tréningu na prístup k dátam. Keďže BERT používa svoju vlastnú funkciu kódovania, budeme musieť definovať funkciu na doplnenie podobnú `padify`, ktorú sme definovali predtým:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

V našom prípade budeme používať predtrénovaný model BERT nazvaný `bert-base-uncased`. Načítajme model pomocou balíka `BertForSequenceClassfication`. To zabezpečí, že náš model už má požadovanú architektúru pre klasifikáciu, vrátane finálneho klasifikátora. Uvidíte varovnú správu, ktorá uvádza, že váhy finálneho klasifikátora nie sú inicializované a model by vyžadoval predtréning - to je úplne v poriadku, pretože presne to sa chystáme urobiť!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Teraz sme pripravení začať tréning! Keďže BERT je už predtrénovaný, chceme začať s pomerne malou rýchlosťou učenia, aby sme nezničili počiatočné váhy.

Všetku náročnú prácu vykonáva model `BertForSequenceClassification`. Keď zavoláme model na tréningových dátach, vráti nám stratu aj výstup siete pre vstupný minibatch. Stratu používame na optimalizáciu parametrov (`loss.backward()` vykonáva spätný prechod) a `out` na výpočet presnosti tréningu porovnaním získaných štítkov `labs` (vypočítaných pomocou `argmax`) s očakávanými `labels`.

Na kontrolu procesu akumulujeme stratu a presnosť počas niekoľkých iterácií a tlačíme ich každých `report_freq` tréningových cyklov.

Tento tréning pravdepodobne zaberie pomerne veľa času, preto obmedzujeme počet iterácií.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Môžete vidieť (najmä ak zvýšite počet iterácií a počkáte dostatočne dlho), že klasifikácia pomocou BERT nám poskytuje celkom dobrú presnosť! Je to preto, že BERT už veľmi dobre rozumie štruktúre jazyka a my potrebujeme iba doladiť záverečný klasifikátor. Avšak, keďže BERT je veľký model, celý proces trénovania trvá dlho a vyžaduje značný výpočtový výkon! (GPU, a ideálne viac ako jeden).

> **Note:** V našom príklade sme používali jeden z najmenších predtrénovaných modelov BERT. Existujú väčšie modely, ktoré pravdepodobne prinesú lepšie výsledky.


## Hodnotenie výkonu modelu

Teraz môžeme vyhodnotiť výkon nášho modelu na testovacej množine údajov. Cyklus hodnotenia je veľmi podobný tréningovému cyklu, ale nemali by sme zabudnúť prepnúť model do hodnotiaceho režimu zavolaním `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Hlavné body

V tejto jednotke sme videli, aké jednoduché je použiť predtrénovaný jazykový model z knižnice **transformers** a prispôsobiť ho na úlohu klasifikácie textu. Podobne je možné modely BERT použiť na extrakciu entít, odpovedanie na otázky a ďalšie úlohy spracovania prirodzeného jazyka.

Modely transformerov predstavujú súčasný špičkový stav v oblasti NLP a vo väčšine prípadov by mali byť prvým riešením, s ktorým začnete experimentovať pri implementácii vlastných NLP riešení. Avšak pochopenie základných princípov rekurentných neurónových sietí, o ktorých sme diskutovali v tomto module, je mimoriadne dôležité, ak chcete vytvárať pokročilé neurónové modely.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
